# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902084


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/86 [00:00<?, ?it/s]

Rendering models:   3%|▎         | 3/86 [00:01<00:52,  1.59it/s]

Rendering models:   5%|▍         | 4/86 [00:03<01:07,  1.21it/s]

Rendering models:   6%|▌         | 5/86 [00:05<01:35,  1.18s/it]

Rendering models:   7%|▋         | 6/86 [00:05<01:13,  1.10it/s]

...

Rendering models:  95%|█████████▌| 82/86 [00:29<00:01,  3.39it/s]

Rendering models:  98%|█████████▊| 84/86 [00:30<00:00,  3.82it/s]

Rendering models:  99%|█████████▉| 85/86 [00:30<00:00,  3.80it/s]

Rendering models: 100%|██████████| 86/86 [00:30<00:00,  2.98it/s]

not-logged-in-b46631f292b19e67bda4      0.010674
not-logged-in-d26daff360c1608523df      0.000133
not-logged-in-621b62a5f3840545c538      0.006567
Thedunkmasta                            0.000210
not-logged-in-0997cfcba8da23453381      0.475591
Thedunkmasta                            0.000201
mlang2017                               0.016056
rcbeck                                  0.000085
sinjinza1984                            0.000081
NikkaAlinea                             0.000185
msalin03                                0.000118
msalin03                                0.000077
msross                                  0.000125
clizardo                                0.000081
mkirms                                  0.000094
mkirms                                  0.000079
M_Mij                                   0.000073
not-logged-in-988154c28a4325297d39      0.000178
djcameron                               0.000071
mchaffin                                0.000119
wthammas            

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())